In [1]:
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict
from sklearn.metrics import f1_score

import utils

H_ONLY='../nlunetwork/results/tuning/optimized/h_only/conf_4/huric/modern_right/'
FN_FT='../nlunetwork/results/tuning/optimized/fn_only_hyper_h/huric/modern_right/'
FN_LU='../nlunetwork/results/tuning/optimized/fn_lu_only_hyper_h/huric/modern_right/'
H_FN_FT='../nlunetwork/results/tuning/optimized/h_and_fn_ft/conf_4/huric/with_framenet_ft/'
H_FN_LU='../nlunetwork/results/tuning/optimized/h_and_fn_lu/conf_4/huric/with_framenet_lu/'

samples_h = utils.load_json(H_ONLY,16)
#samples_fn_ft = utils.load_json(FN_FT, 0)
samples_fn = utils.load_json(FN_LU, 0)
#samples_h_fn_ft = utils.load_json(H_FN_FT,48)
samples_h_fn = utils.load_json(H_FN_LU,25)

# also the XML stuff
HURIC_LOCATION = '../data/huric/modern/source'
gold_attn = utils.load_attention_gold('../data/huric/modern_right/lu_disc_final.tsv')

In [2]:
def eval_align_array(pred_attn_vals, gold_one_hot):
    # how many targets are there (non-zero elements)
    n_targets = sum(gold_one_hot)
    # take n_targets best values from the predicted attention values
    if n_targets:
        pred_target_indexes = np.argpartition(pred_attn_vals, -n_targets)[-n_targets:]
    else:
        # just be careful when n_targets is zero, argpartition would return all the indexes
        pred_target_indexes = []
    # now from the indexes found, get the 0/1 representation
    pred_targets = [1 if el in pred_target_indexes else 0 for el in range(len(pred_attn_vals))]
    #print(n_targets, pred_target_indexes, gold_one_hot, pred_targets)
    # and get the f1 score only on the 1 class (average is binary)    
    f1 = f1_score(gold_one_hot, pred_targets, average='binary')
    return f1

def evaluate_attn(samples, golds):
    total = defaultdict(lambda: 0)
    total_only_correct = defaultdict(lambda: 0)
    correct_frame_cnt = 0
    for s in samples:
        #print(s['id'])
        gold = golds[str(s['id'])]
        for what in ['lu', 'lu+disc', 'lu+disc2']:
            gold_one_hot = gold[what]
            measure = eval_align_array(s['intent_attentions'], gold_one_hot)
            total[what] += measure
        if gold['frame'] == s['intent_pred']:
            correct_frame_cnt += 1
            for what in ['lu', 'lu+disc', 'lu+disc2']:
                gold_one_hot = gold[what]
                measure = eval_align_array(s['intent_attentions'], gold_one_hot)
                total_only_correct['only_correct_frame_{}'.format(what)] += measure
    for what, val in total.items():
        print(what, val/len(samples))
    for what, val in total_only_correct.items():
        print(what, val/correct_frame_cnt)
        

In [3]:
evaluate_attn(samples_h, gold_attn)

lu 0.0636998254799302
lu+disc 0.1273996509598604
lu+disc2 0.14761489237929026
only_correct_frame_lu 0.0625
only_correct_frame_lu+disc 0.12409420289855072
only_correct_frame_lu+disc2 0.1444746376811594


In [4]:
evaluate_attn(samples_fn, gold_attn)

lu 0.9511343804537522
lu+disc 0.8685282140779523
lu+disc2 0.8454043048283887
only_correct_frame_lu 0.9923076923076923
only_correct_frame_lu+disc 0.8884615384615384
only_correct_frame_lu+disc2 0.8867521367521367


In [5]:
evaluate_attn(samples_h_fn, gold_attn)

lu 0.44851657940663175
lu+disc 0.45898778359511344
lu+disc2 0.4714950552646888
only_correct_frame_lu 0.4601593625498008
only_correct_frame_lu+disc 0.4677954847277556
only_correct_frame_lu+disc2 0.48240371845949526
